In [15]:
import numpy as np
import glob
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from sklearn.metrics import accuracy_score
import copy
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

In [16]:
birds_labels = {
    "other": 0,
    "comcuc": 1,
    "cowpig1": 2,
    "eucdov": 3,
    "eueowl1": 4,
    "grswoo": 5,
    "tawowl1": 6
}

birds = [bird for bird in list(birds_labels.keys()) if bird != "other"]

In [17]:
def unique_rows(matrix):
    indexes = []
    labels = []
    
    for i, row in enumerate(matrix):
        if np.unique(row).size == 1:
            indexes.append(i)
            labels.append(np.unique(row)[0])
    
    indexes = np.array(indexes)
    labels = np.array(labels)
    
    return indexes, labels

In [18]:
def load_data(bird):
    labels = []
    features = []
    bird_id = birds_labels[bird]
    
    path = f'./data/{bird}/'
    labels_files = glob.glob(path + '*labels.npy')
    counter = None
    
    for i, file in enumerate(labels_files):
        print(f'{bird}: {i + 1}/{len(labels_files)}', end='\r')
        counter = i
        data_id = path + ''.join(file.split(".labels.npy")).split('/')[-1] + '.npy'
        
        annotations = np.load(file)
        feature = np.load(data_id)
        
        ind, label = unique_rows(annotations)
        
        if len(ind) == 0:
            continue
        
        labels.append(label)
        features.append(feature[ind])

    print('\n')
    labels = np.concatenate(labels)
    features = np.concatenate(features)
    
    return labels, features

In [19]:
X = []
y = []

for bird in birds:
    labels, features = load_data(bird)
    X.append(features)
    y.append(labels)
    
X = np.concatenate(X)
y = np.concatenate(y)

comcuc: 200/200

cowpig1: 200/200

eucdov: 200/200

eueowl1: 200/200

grswoo: 200/200

tawowl1: 200/200



In [20]:
# Apply MinMaxScaler to make the data non-negative
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# Perform feature selection
k_best_selector = SelectKBest(chi2, k=32)  # Select 2 best features
X_selected = k_best_selector.fit_transform(X, y)

# Print the selected features
selected_feature_indices = k_best_selector.get_support(indices=True)
# selected_feature_names = [data.feature_names[i] for i in selected_feature_indices]
# print("Selected Features:", selected_feature_names)

In [21]:
X = X_selected
X.shape

(85500, 32)

# Gradient Boosting

In [22]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [ ]:
# Create a Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier()

# Perform k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store accuracy scores for each fold



n_estimators = [100, 200, 300] # Number of boosting stages
learning_rate = [0.1, 0.01, 0.001] # Learning rate
max_depth = [3, 4, 5]  # Maximum depth of each tree


for n in n_estimators:
    for l in learning_rate:
        for m in max_depth:
            gb_classifier = GradientBoostingClassifier(n_estimators=n, learning_rate=l, max_depth=m)
            print(f"n = {n}, lr = {l}, m = {m}")
            accuracy_scores = []
            
            # Iterate over each fold
            for i, (train_index, test_index) in enumerate(kfold.split(X)):
                # Split the data into train and test sets for the fold
                print(f"Fold = {i}")
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]

                # Fit the Gradient Boosting Classifier on the training data
                gb_classifier.fit(X_train, y_train)

                # Predict the labels for the test data
                y_pred = gb_classifier.predict(X_test)

                # Calculate the accuracy score for the fold
                accuracy = accuracy_score(y_test, y_pred)

                # Store the accuracy score
                accuracy_scores.append(accuracy)
            print(f"Accuracy {np.mean(accuracy_scores)}")

# Calculate the average accuracy across all folds
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)

# Print the average accuracy
print("Average Accuracy:", average_accuracy)


n = 100, lr = 0.1, m = 3
Fold = 0
Fold = 1
Fold = 2
Fold = 3
Fold = 4
Accuracy 0.9278011695906432
n = 100, lr = 0.1, m = 4
Fold = 0
Fold = 1
Fold = 2
Fold = 3
Fold = 4
Accuracy 0.9321637426900585
n = 100, lr = 0.1, m = 5
Fold = 0
Fold = 1
Fold = 2
Fold = 3
Fold = 4
Accuracy 0.934608187134503
n = 100, lr = 0.01, m = 3
Fold = 0
Fold = 1
Fold = 2
Fold = 3
Fold = 4
Accuracy 0.8908070175438597
n = 100, lr = 0.01, m = 4
Fold = 0
Fold = 1
Fold = 2
Fold = 3
Fold = 4
Accuracy 0.9011345029239767
n = 100, lr = 0.01, m = 5
Fold = 0
Fold = 1
Fold = 2
Fold = 3
